In [1]:
import spacy #for tokenization and lemmatization
nlp = spacy.load('en_core_web_sm')
lemmatizer = nlp.get_pipe("lemmatizer")

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn #to use WordNet

import re #regular expression for removing punctuations
import requests #read the British to American dictionary
import pandas as pd
import csv

[nltk_data] Downloading package wordnet to /home/jinhan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
british_to_american = requests.get(url).json()

In [13]:
root = "/mnt/h/OneDrive - The University Of Hong Kong/bdd/explanations/"
file = "exp_vehicle_bdd.csv"
import os

exp = pd.read_csv(os.path.join(root, file)) #Read the file with explanation text
# Label: vehicle, human
# Category: person, rider, car, bus, truck
exp = exp.astype({'text':'string','image':'string','label':'string','category':'string'})
exp.head()

,Unnamed: 0,subject_no,Trial_Index_,block,text,block_sequence,block_name,block_num,path,image,trialID,label,category,number,lastBlockType
0,0,1,1,2,he is walking with legs,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,411.jpg,1,human,person,NaN,NaN
1,1,1,2,2,slim with legs and hand,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,608.jpg,2,human,person,NaN,NaN
2,2,1,3,2,human face,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,480.jpg,3,human,person,NaN,NaN
3,3,1,4,2,human cloth with hat,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,85.jpg,4,human,person,NaN,NaN
4,4,1,5,2,human back with t shirt,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,703.jpg,5,human,person,NaN,NaN


In [15]:
def preprocess(row):
    #TODO: how to handle trials skipped with empty explanation? shouldn't count it in when calculating score mean for this subject (Fine if we get 0 scores which has no effect when taking average?)
    if pd.isna(row['text']): return []

    # Clean trials where subject answer they didn't see the image
    for keywords in ["can't see","cant see",'barely see','cannot see','no see',"not see", "dont see", "hardly see", 'cammot see',"didnt see","dont really see","seems nobody","cnt see"]:
        if keywords in row['text'].lower():
            return []  

    string = re.sub(r'[^\w\s]', '', row['text']) #Remove punctuations
    sentence = nlp(string) #Apply Spacy
    lemma_list = [token.lemma_ for token in sentence] #Lemmatization
    am_list = [] #Empty list to put changed spellings
    for word in lemma_list:
        if word in british_to_american:
            am_list.append(british_to_american[word]) #Replace with American spelling if there is a difference
        else:
            am_list.append(word) #Just append the original word if there is no difference
    return am_list

exp['tokenized'] = exp.apply(lambda row: preprocess(row), axis = 1) #Process explanations
exp.head()

,Unnamed: 0,subject_no,Trial_Index_,block,text,block_sequence,block_name,block_num,path,image,trialID,label,category,number,lastBlockType,tokenized
0,0,1,1,2,he is walking with legs,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,411.jpg,1,human,person,NaN,NaN,"[he, be, walk, with, leg]"
1,1,1,2,2,slim with legs and hand,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,608.jpg,2,human,person,NaN,NaN,"[slim, with, leg, and, hand]"
2,2,1,3,2,human face,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,480.jpg,3,human,person,NaN,NaN,"[human, face]"
3,3,1,4,2,human cloth with hat,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,85.jpg,4,human,person,NaN,NaN,"[human, cloth, with, hat]"
4,4,1,5,2,human back with t shirt,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,703.jpg,5,human,person,NaN,NaN,"[human, back, with, t, shirt]"


In [16]:
def get_rating(item):
    rating = {}
    with open('./Jennifer/sensorimotor.csv') as file: #Read the file with sensorimotor strength ratings
        reader = csv.DictReader(file)
        for row in reader:
            rating[row['word']] = float(row[item]) #Convert ratings into dictionaries
    return rating

visual_dict = get_rating('visual')

In [17]:
def avg_rating(row, dict_name): #Calculate average rating per word for each explanation
    rating_list = []
    for lemma in row['tokenized']:
        lemma = lemma.upper()
        if lemma in dict_name:
            rating_list.append(dict_name[lemma])
    if len(rating_list) != 0:
        return sum(rating_list)/len(rating_list)
    else:
        return 0

In [ ]:
max(visual_dict.values())

5.0

In [18]:
exp['visual'] = exp.apply(lambda row: avg_rating(row, visual_dict), axis = 1) #Get visual strength for each explanation

exp.head()

,Unnamed: 0,subject_no,Trial_Index_,block,text,block_sequence,block_name,block_num,path,image,trialID,label,category,number,lastBlockType,tokenized,visual
0,0,1,1,2,he is walking with legs,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,411.jpg,1,human,person,NaN,NaN,"[he, be, walk, with, leg]",3.197386
1,1,1,2,2,slim with legs and hand,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,608.jpg,2,human,person,NaN,NaN,"[slim, with, leg, and, hand]",3.422308
2,2,1,3,2,human face,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,480.jpg,3,human,person,NaN,NaN,"[human, face]",4.836111
3,3,1,4,2,human cloth with hat,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,85.jpg,4,human,person,NaN,NaN,"[human, cloth, with, hat]",4.298203
4,4,1,5,2,human back with t shirt,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,703.jpg,5,human,person,NaN,NaN,"[human, back, with, t, shirt]",3.950868


In [21]:
exp.loc[12602]

Unnamed: 0                                                  13
subject_no                                                  31
Trial_Index_                                                14
block                                                        3
text                                                CA N'T SEE
block_sequence                                             3.0
block_name                                            exp_car2
block_num                                                    1
path              /Veh_ExpTask_Test1_car_deploy/results/031ET2
image                                                  902.jpg
trialID                                                     14
label                                                  vehicle
category                                                   car
number                                                     NaN
lastBlockType                                              NaN
tokenized                                        [CA, N

In [22]:
print("Max visual score\n")

print(exp["visual"].max())

exp.loc[exp["visual"].idxmax()]

Max visual score

5.0


Unnamed: 0                                                  13
subject_no                                                  31
Trial_Index_                                                14
block                                                        3
text                                                CA N'T SEE
block_sequence                                             3.0
block_name                                            exp_car2
block_num                                                    1
path              /Veh_ExpTask_Test1_car_deploy/results/031ET2
image                                                  902.jpg
trialID                                                     14
label                                                  vehicle
category                                                   car
number                                                     NaN
lastBlockType                                              NaN
tokenized                                        [CA, N

In [ ]:
[visual_dict[a.upper()] for a in ['see','the','head']]
visual_dict['VIEW']

4.95

In [ ]:
mask = exp["category"].str.contains("truck")
exp[mask]

,Unnamed: 0,subject_no,Trial_Index_,block,text,block_sequence,block_name,block_num,path,image,trialID,label,category,number,lastBlockType,tokenized,visual,wordnet_similarity_label,wordnet_similarity_category
18480,0,10,1,5,light reflecting,NaN,NaN,4,/Veh_ExpTask_Test4_bustruck_deploy/results/001...,914.jpg,149,vehicle,truck,9.0,5.0,"[light, reflect]",4.050000,0.062500,0.052632
18481,1,10,2,5,white color,NaN,NaN,4,/Veh_ExpTask_Test4_bustruck_deploy/results/001...,940.jpg,150,vehicle,truck,9.0,5.0,"[white, color]",4.772222,0.094017,0.072917
18482,2,10,3,5,reflecting light,NaN,NaN,4,/Veh_ExpTask_Test4_bustruck_deploy/results/001...,559.jpg,151,vehicle,truck,9.0,5.0,"[reflect, light]",4.050000,0.062500,0.052632
18483,3,10,4,5,"reflecting light, car plate",NaN,NaN,4,/Veh_ExpTask_Test4_bustruck_deploy/results/001...,1100.jpg,152,vehicle,truck,9.0,5.0,"[reflect, light, car, plate]",4.234722,0.129167,0.158958
18484,4,10,5,5,shadow of an object,NaN,NaN,4,/Veh_ExpTask_Test4_bustruck_deploy/results/001...,222.jpg,153,vehicle,truck,9.0,5.0,"[shadow, of, an, object]",2.537118,0.097222,0.072125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19183,7,60,8,5,truck door,NaN,NaN,4,/Veh_ExpTask_Test4_bustruck_deploy/results/060ET4,914.jpg,156,vehicle,truck,9.0,5.0,"[truck, door]",4.462418,0.155556,0.541667
19184,8,60,9,5,"top of the red car front, above the drive seat",NaN,NaN,4,/Veh_ExpTask_Test4_bustruck_deploy/results/060ET4,222.jpg,157,vehicle,truck,9.0,5.0,"[top, of, the, red, car, front, above, the, dr...",3.085335,0.098352,0.103400
19185,9,60,10,5,grey box,NaN,NaN,4,/Veh_ExpTask_Test4_bustruck_deploy/results/060ET4,852.jpg,158,vehicle,truck,9.0,5.0,"[gray, box]",4.495021,0.133333,0.111111
19186,10,60,11,5,higher red lights,NaN,NaN,4,/Veh_ExpTask_Test4_bustruck_deploy/results/060ET4,833.jpg,159,vehicle,truck,9.0,5.0,"[high, red, light]",4.107407,0.068697,0.056896


In [23]:
mask = exp["text"].str.contains("see")
exp[mask].to_csv('rows_with_see1.csv')

In [24]:
wn_lemmas = set(wn.all_lemma_names()) #Get all lemmas in WordNet

def avg_similarity(row, col): #Calculate average similarity to the label for each explanation
    sim_list = []
    label = row[col]
    # if label == 'Tennis Ball': #Dealing with the special case of the two-word label, tennis ball
    #     label = 'tennis_ball'
    for lemma in row['tokenized']:
        if lemma in wn_lemmas: #Check if the word is in WordNet
            sim = wn.synsets(label)[0].path_similarity(wn.synsets(lemma)[0],simulate_root = False) #Disable simulate root so that we only get similarities for nouns
            if sim is not None: #For other parts of speech, similarity will be empty
                sim_list.append(sim)
    if len(sim_list) != 0:
        return sum(sim_list)/len(sim_list)
    else:
        return 0

In [25]:
exp['wordnet_similarity_label'] = exp.apply(lambda row: avg_similarity(row, 'label'), axis = 1) #Get WordNet similarity for each explanation
exp['wordnet_similarity_category'] = exp.apply(lambda row: avg_similarity(row, 'category'), axis = 1) #Get WordNet similarity for each explanation
exp.head()

,Unnamed: 0,subject_no,Trial_Index_,block,text,block_sequence,block_name,block_num,path,image,trialID,label,category,number,lastBlockType,tokenized,visual,wordnet_similarity_label,wordnet_similarity_category
0,0,1,1,2,he is walking with legs,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,411.jpg,1,human,person,NaN,NaN,"[he, be, walk, with, leg]",3.197386,0.051964,0.111195
1,1,1,2,2,slim with legs and hand,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,608.jpg,2,human,person,NaN,NaN,"[slim, with, leg, and, hand]",3.422308,0.051316,0.105556
2,2,1,3,2,human face,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,480.jpg,3,human,person,NaN,NaN,"[human, face]",4.836111,0.527778,0.112500
3,3,1,4,2,human cloth with hat,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,85.jpg,4,human,person,NaN,NaN,"[human, cloth, with, hat]",4.298203,0.379808,0.125926
4,4,1,5,2,human back with t shirt,2.0,exp_car1,1,/Hum_ExpTask_Test1_deploy/results/001ET1,703.jpg,5,human,person,NaN,NaN,"[human, back, with, t, shirt]",3.950868,0.292236,0.111219


Should the explanation with original word be excluded?